## Create Operations Rehearsal Blocks from and Opsim data base

Take a OpsSim database created by the simulated scheduler for the 3rd operations rehearsal and produce blocks for the for the scheduler CSC.

CWW - March 2024

In [10]:
%config InlineBackend.figure_format = 'retina'

from lsst.ts.observing import ObservingBlock, ObservingScript

from sqlalchemy import create_engine

import pandas as pd

import matplotlib
import matplotlib.pyplot as pyplot

Let's get the data base into a Pandas DataFrame.  We will use sqlalchemy to make a database connection and then pandas to read the database tables into DataFrames.  This could have SQL selections in it but I am just reading the whole table in.

In [11]:
engine = create_engine('sqlite:////sdf/data/rubin/shared/ops-rehearsal-3/scheduler_sims/ops_rehearsal_apr_2024_v3_playbackBlock.db')
Observations = pd.read_sql_table('observations', engine)
Observations.head()

,observationId,fieldRA,fieldDec,observationStartMJD,flush_by_mjd,visitExposureTime,filter,rotSkyPos,rotSkyPos_desired,numExposures,...,sunRA,sunDec,moonRA,moonDec,moonDistance,solarElong,moonPhase,cummTelAz,scripted_id,playbackBlock
0,0,125.122874,-14.886583,60390.000795,60390.047222,30.0,r,137.587154,0.0,1,...,0.484832,0.209961,138.272151,21.734606,38.825500,123.384207,74.091528,48.984655,0,0
1,1,124.877453,-15.117924,60390.001195,60390.047222,30.0,r,139.225638,0.0,1,...,0.485197,0.210119,138.275168,21.733334,39.123316,123.110757,74.093018,48.922431,0,0
2,2,124.812108,-15.067734,60390.001593,60390.047222,30.0,r,140.648646,0.0,1,...,0.485559,0.210276,138.278161,21.732069,39.097540,123.056892,74.094496,48.699008,0,0
3,3,125.031639,-14.886427,60390.001993,60390.047222,30.0,r,141.834165,0.0,1,...,0.485923,0.210434,138.281169,21.730796,38.854555,123.296454,74.095982,48.810952,0,0
4,4,124.878561,-14.854945,60390.002392,60390.047222,30.0,r,143.370535,0.0,1,...,0.486287,0.210592,138.284166,21.729524,38.875252,123.155676,74.097463,48.443917,0,0


In [12]:
# Summarize and then select columns we want to use

print(Observations.columns)

# Make a df with selected values.

visit_info = Observations[['observationId', 'fieldRA', 'fieldDec', 
    'observationStartMJD',
    'filter', 'rotSkyPos', 'rotTelPos', 
    'visitTime', 'visitExposureTime',
    'altitude', 'azimuth', 'slewTime',
    'note', 'target', 'fieldId',
    'block_id', 'playbackBlock']]

Index(['observationId', 'fieldRA', 'fieldDec', 'observationStartMJD',
       'flush_by_mjd', 'visitExposureTime', 'filter', 'rotSkyPos',
       'rotSkyPos_desired', 'numExposures', 'airmass', 'seeingFwhm500',
       'seeingFwhmEff', 'seeingFwhmGeom', 'skyBrightness', 'night', 'slewTime',
       'visitTime', 'slewDistance', 'fiveSigmaDepth', 'altitude', 'azimuth',
       'paraAngle', 'cloud', 'moonAlt', 'sunAlt', 'note', 'target', 'fieldId',
       'proposalId', 'block_id', 'observationStartLST', 'rotTelPos',
       'rotTelPos_backup', 'moonAz', 'sunAz', 'sunRA', 'sunDec', 'moonRA',
       'moonDec', 'moonDistance', 'solarElong', 'moonPhase', 'cummTelAz',
       'scripted_id', 'playbackBlock'],
      dtype='object')


In [13]:
#Print out the BLOCK-IDs.
pd.unique(visit_info.playbackBlock)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [14]:
#Group the data frame by block and print how many are in each block.
BLOCKS = visit_info.groupby(['playbackBlock'])
BLOCKS.observationId.count()

playbackBlock
0      60
1      60
2      60
3      60
4      60
       ..
340    60
341    60
342    26
343    60
344     1
Name: observationId, Length: 345, dtype: int64

### Extract the data frames we will use for the blocks.

In [15]:
#Get BLOCK = 0

selected_block = 0
block = BLOCKS.get_group( (selected_block,) )

In [16]:
# Create three extracted data frames with the correct field names.  We will 
# iterate over them later.  Also add some constant values to the data frame.
# one is for the sleep command (for the slew), two are for the take_image command.

slew_block = block[['slewTime']]
slew_block.columns = ['sleep_for']

obs_block = block[['visitExposureTime', 'target', 'filter']].copy()
obs_block.columns = ['exp_times', 'note', 'filter']
obs_block['filter'] = obs_block['filter'].replace(['i', 'r', 'g'], ['i_06', 'r_03', 'g_07'])

obs_block.insert(len(obs_block.columns), 'image_type', 'OBJECT')
obs_block.insert(len(obs_block.columns), 'nimages', 1)

visit_metadata = block[['fieldRA', 'fieldDec', 'rotSkyPos']]
visit_metadata.columns = ['ra', 'dec', 'rot_sky']

# Print out the first five rows of the obs and sleep data frames
print("obs_block")
display(obs_block.head())
print("Visit Metadata")
display(visit_metadata.head())
print("Slewing info")
display(slew_block.head())

obs_block


,exp_times,note,filter,image_type,nimages
0,30.0,Rubin_SV_125_-15,r_03,OBJECT,1
1,30.0,Rubin_SV_125_-15,r_03,OBJECT,1
2,30.0,Rubin_SV_125_-15,r_03,OBJECT,1
3,30.0,Rubin_SV_125_-15,r_03,OBJECT,1
4,30.0,Rubin_SV_125_-15,r_03,OBJECT,1


Visit Metadata


,ra,dec,rot_sky
0,125.122874,-14.886583,137.587154
1,124.877453,-15.117924,139.225638
2,124.812108,-15.067734,140.648646
3,125.031639,-14.886427,141.834165
4,124.878561,-14.854945,143.370535


Slewing info


,sleep_for
0,68.656437
1,3.614541
2,3.357309
3,3.561699
4,3.457968


### Produce the scripts and blocks

In [17]:
# Make a function to print the json structure in a multi-line human readable 
# form for the blocks GitHub Repository.

def pretty_print_model(model):
    print(model.model_dump_json(indent=4))

In [18]:
# Set how many rows of the BLOCK to process
rows = obs_block.shape[0]

# All the scripts in the block will be appended to this list.
script_list = []

# First write the script that puts us into playlist mode.
playlist_name = f'block-{selected_block}_playlist_{rows}'

playlist_script = ObservingScript(name="run_command.py",  standard=True,
            parameters={"component": "CCCamera", "cmd": "play", 
                        "parameters": {"playlist": playlist_name, "repeat": False}})

script_list.append(playlist_script)

for row in range(rows):

    # Convert this row of the data frames parameters into dictionaries. This line
    # converts the row of the data frame to a dictionary, then selects the first
    # element of the list of dictionaries.
    slew_dict = slew_block.iloc[[row],:].to_dict('records')[0]
    
    # For the obs information, we need to add the visit metadata to a separate 
    # dictionary to add to the obs dictionary as a nested entry.
    obs_dict = obs_block.iloc[[row],:].to_dict('records')[0]
    metadata_dict = visit_metadata.iloc[[row],:].to_dict('records')[0]
    obs_dict["visit_metadata"] = metadata_dict

    # Place the the sleep parameters into a script
    sleep_script = ObservingScript(name="sleep.py",  standard=True,
            parameters= slew_dict)
    
    # Place the the ComCam image parameters into a script
    take_image_script = ObservingScript(name="take_image_comcam.py", standard=True, 
                                        parameters = obs_dict)

    # Append them to the running script list    
    script_list.append(sleep_script)
    script_list.append(take_image_script)

# Build the block dictionary
observing_block = ObservingBlock(name=f'BLOCK-{selected_block:03d}', 
                                 program="ops-rehearsal-3", scripts=script_list)

pretty_print_model(observing_block)

{
    "name": "BLOCK-000",
    "id": "6ffcb98e-287a-40db-9c64-a079f3cce70a",
    "program": "ops-rehearsal-3",
    "constraints": [],
    "scripts": [
        {
            "name": "run_command.py",
            "standard": true,
            "parameters": {
                "component": "CCCamera",
                "cmd": "play",
                "parameters": {
                    "playlist": "block-0_playlist_60",
                    "repeat": false
                }
            }
        },
        {
            "name": "sleep.py",
            "standard": true,
            "parameters": {
                "sleep_for": 68.65643692795837
            }
        },
        {
            "name": "take_image_comcam.py",
            "standard": true,
            "parameters": {
                "exp_times": 30.0,
                "note": "Rubin_SV_125_-15",
                "filter": "r_03",
                "image_type": "OBJECT",
                "nimages": 1,
                "visit_metadata": {
  